In [28]:

#    Multi-Disciplinary Design Optimisation of a Hydrogen Powered Commercial Aircraft (MDOHPCA)
#    this is the master file that runs each of the individual disciplines 
#    It sets up each of the analyses, then converts it into an openMDAO problem with a certain structure, then solves for minimum mass



In [29]:

import pandas as pd
import numpy as np
import openmdao.api as om

In [30]:
class MDOHPCA(om.Group):
    """
    Top level group containing the MDO.
    """           
    def setup(self):
        """
        set up each person's disciplines that can contain analyses
        """
        self.add_subsystem('aero', aero())
        self.add_subsystem('sys', sys())
        self.add_subsystem('stab', stab())
        self.add_subsystem('struct', struct())
        
        self.add_subsystem('mass', mass())
        
    def configure(self):
        #promote all variables (lazy option, they can be connected individually)
        self.promotes('sys',any=['*'])
        self.promotes('struct',any=['*'])
        
        self.promotes('mass',any=['*'])

In [31]:
class mass(om.ExplicitComponent):
    def setup(self):
        self.add_input('systems_mass')
        self.add_input('weight_structures')
        
        self.add_output('m')
        prob.model.add_objective('m')
        
        #prob.model.add_constraint('m', lower = 20000)

    def compute(self,inputs,outputs):
        outputs['m'] = inputs['systems_mass'] + inputs['weight_structures']

In [32]:
%run struct/structsetup.ipynb
#%run stab/stabsetup.ipynb
%run systems/systemssetup.ipynb
class aero(om.ExplicitComponent):
    pass
class stab(om.ExplicitComponent):
    pass

In [33]:
prob = om.Problem(model = MDOHPCA())
#prob.model.add_subsystem('aero', om.Group())

In [34]:
#setting inputs to reduce ambiguities where inputs differ in different disciplines
prob.model.set_input_defaults('CL', val = 0.6)
prob.model.set_input_defaults('sweep', val = 15)
prob.model.set_input_defaults('c', val = 4)
prob.model.set_input_defaults('b', val = 60)
prob.model.set_input_defaults('root_x', val = 10)
prob.model.set_input_defaults('m', val = 240000)


prob.model.add_design_var('fuselage_diameter')

In [35]:
#declare problem options (driver, optimiser)
prob.model.driver = om.ScipyOptimizeDriver()
prob.model.driver.options['optimizer'] = 'SLSQP'
#prob.nonlinear_solver = om.NonlinearBlockGS()
#prob.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
prob.model.nonlinear_solver = om.NonlinearBlockGS()
prob.model.nonlinear_solver.options['iprint'] = 2
prob.model.nonlinear_solver.options['maxiter'] = 50

#prob.nonlinear_solver.options['maxiter'] = 100
prob.model.driver.options['maxiter'] = 5
prob.model.driver.options['tol'] = 1e-8

#setup problem to be run
prob.setup()

In [36]:
#%run aero/aerosetup.ipynb

In [37]:
prob.run_model()

IndexError: 'struct.optimisation' <class optimisation>: Error calling compute(), index 0 is out of bounds for axis 0 with size 0

In [ ]:
prob.model.list_outputs()

In [ ]:
prob.model.get_constraints()

In [ ]:
#get properties of specific constants in disciplines
prob.model.sys.options._dict['pp_mass']